In [50]:
import numpy as np

MAT_SIZE=32
PE_MAT_SIZE=8

MEM_ORD_ADDR=3*MAT_SIZE*MAT_SIZE
MEM_TMP_ADDR=6*MAT_SIZE*MAT_SIZE
PE_COUNT=MAT_SIZE//PE_MAT_SIZE
MEM_LENGTH=MEM_TMP_ADDR+2*PE_COUNT*PE_MAT_SIZE*PE_MAT_SIZE

# mem=np.zeros(PE_MAT_SIZE*PE_MAT_SIZE*MAT_COUNT,np.uint32)
mem=np.zeros(MEM_LENGTH,np.uint32)
mem[:2*MAT_SIZE*MAT_SIZE]=np.arange(2*MAT_SIZE*MAT_SIZE)


class PE:
    def test():
        print("test")
    def mul(m1_ptr,m2_ptr,ans_ptr,mat_size=PE_MAT_SIZE):
        mem[ans_ptr:ans_ptr+mat_size*mat_size]=np.reshape(np.matmul(
            np.reshape(mem[m1_ptr:m1_ptr+mat_size*mat_size],(mat_size,mat_size)),
            np.reshape(mem[m2_ptr:m2_ptr+mat_size*mat_size],(mat_size,mat_size))),
            mat_size*mat_size)

    def add(m1_ptr,m2_ptr,ans_ptr,mat_size=PE_MAT_SIZE):
        mem[ans_ptr:ans_ptr+mat_size*mat_size]=np.add(
            mem[m1_ptr:m1_ptr+mat_size*mat_size],
            mem[m2_ptr:m2_ptr+mat_size*mat_size])

class Display:
    def mat(ptr,mat_size=PE_MAT_SIZE):
        for i in range(ptr,ptr+mat_size*mat_size,mat_size):
            for d in mem[i:i+mat_size]:
                str_d=str(d)
                print(" "*(2-len(str_d))+str_d,end=' ')
            print()
        print()

    def all(base=0,mat_size=PE_MAT_SIZE):
        for ptr in range(base,MEM_LENGTH,mat_size*mat_size):
            Display.mat(ptr,mat_size)
            print()

mat1_ptr=0
mat2_ptr=mat1_ptr+MAT_SIZE*MAT_SIZE
matAns_ptr=mat2_ptr+MAT_SIZE*MAT_SIZE

mat1_ord_ptr=MEM_ORD_ADDR+mat1_ptr
mat2_ord_ptr=MEM_ORD_ADDR+mat2_ptr
matAns_ord_ptr=MEM_ORD_ADDR+matAns_ptr

# intialize tmp zero matrix
# for PE_i in range(PE_COUNT):
#     addr=MEM_TMP_ADDR+PE_i*2*PE_MAT_SIZE*PE_MAT_SIZE
#     mem[addr:addr+PE_MAT_SIZE*PE_MAT_SIZE]=np.zeros(PE_MAT_SIZE*PE_MAT_SIZE)

# #test PE
# m1_ptr=0
# m2_ptr=m1_ptr+PE_MAT_SIZE*PE_MAT_SIZE
# ans_ptr=m2_ptr+PE_MAT_SIZE*PE_MAT_SIZE

# PE.add(m1_ptr,m2_ptr,ans_ptr)

# Display.mat(m1_ptr)
# Display.mat(m2_ptr)

# Display.mat(ans_ptr)
# #test PE

# def get_addr(i,j,base=0):
#     return i*PE_MAT_SIZE*MAT_SIZE+j*PE_MAT_SIZE+base


def order_mat(source,target):      
    for i in range(MAT_SIZE//PE_MAT_SIZE):
        for j in range(MAT_SIZE//PE_MAT_SIZE):
            start=i*PE_MAT_SIZE*MAT_SIZE+j*PE_MAT_SIZE
            # print((i,j),start)
            for k in range(PE_MAT_SIZE):
                for l in range(PE_MAT_SIZE):
                    addr=start+k*MAT_SIZE+l
                    mem[target]=mem[source+addr]
                    target+=1
   
def reorder_mat(source,target):      
    for i in range(MAT_SIZE//PE_MAT_SIZE):
        for j in range(MAT_SIZE//PE_MAT_SIZE):
            start=i*PE_MAT_SIZE*MAT_SIZE+j*PE_MAT_SIZE
            # print((i,j),start)
            for k in range(PE_MAT_SIZE):
                for l in range(PE_MAT_SIZE):
                    addr=start+k*MAT_SIZE+l
                    mem[target+addr]=mem[source]
                    source+=1
                
order_mat(mat1_ptr,mat1_ord_ptr)
order_mat(mat2_ptr,mat2_ord_ptr)

# Display.mat(mat1_ptr,MAT_SIZE)
# reorder_mat(mat1_ord_ptr,matAns_ptr)
# Display.mat(mat1_ord_ptr,MAT_SIZE)
# Display.mat(matAns_ptr,MAT_SIZE)
   
# # print(mem[matAns_ptr:matAns_ptr+MAT_SIZE*MAT_SIZE])

def get_addr(i,j,base=0):
    return i*PE_MAT_SIZE*MAT_SIZE+j*PE_MAT_SIZE*PE_MAT_SIZE+base


for i in range(MAT_SIZE//PE_MAT_SIZE):    
    for PE_i in range(PE_COUNT):
        add_addr=MEM_TMP_ADDR+PE_i*2*PE_MAT_SIZE*PE_MAT_SIZE
        mul_addr=add_addr+PE_MAT_SIZE*PE_MAT_SIZE

        # clear add matirx
        mem[add_addr:add_addr+PE_MAT_SIZE*PE_MAT_SIZE]=np.zeros(PE_MAT_SIZE*PE_MAT_SIZE)

        for j in range(MAT_SIZE//PE_MAT_SIZE):
            PE.mul(get_addr(i,j,mat1_ord_ptr),get_addr(j,PE_i,mat2_ord_ptr),mul_addr)
            PE.add(add_addr,mul_addr,add_addr)
            # print(get_addr(i,j),get_addr(j,PE_i))
        # Display.mat(add_addr)
        ans_addr=matAns_ord_ptr+get_addr(i,PE_i)
        mem[ans_addr:ans_addr+PE_MAT_SIZE*PE_MAT_SIZE]=mem[add_addr:add_addr+PE_MAT_SIZE*PE_MAT_SIZE]        
        
reorder_mat(matAns_ord_ptr,matAns_ptr)
# Display.mat(matAns_ptr,MAT_SIZE)

ans1=np.reshape(mem[matAns_ptr:matAns_ptr+MAT_SIZE*MAT_SIZE],(MAT_SIZE,MAT_SIZE))

mat1=np.reshape(mem[mat1_ptr:mat1_ptr+MAT_SIZE*MAT_SIZE],(MAT_SIZE,MAT_SIZE))
mat2=np.reshape(mem[mat2_ptr:mat2_ptr+MAT_SIZE*MAT_SIZE],(MAT_SIZE,MAT_SIZE))

ans2=np.matmul(mat1,mat2)

print(ans1)
print(ans2)
print(np.array_equal(ans1,ans2))

[[  841216   841712   842208 ...   855600   856096   856592]
 [ 2397696  2399216  2400736 ...  2441776  2443296  2444816]
 [ 3954176  3956720  3959264 ...  4027952  4030496  4033040]
 ...
 [45979136 46009328 46039520 ... 46854704 46884896 46915088]
 [47535616 47566832 47598048 ... 48440880 48472096 48503312]
 [49092096 49124336 49156576 ... 50027056 50059296 50091536]]
[[  841216   841712   842208 ...   855600   856096   856592]
 [ 2397696  2399216  2400736 ...  2441776  2443296  2444816]
 [ 3954176  3956720  3959264 ...  4027952  4030496  4033040]
 ...
 [45979136 46009328 46039520 ... 46854704 46884896 46915088]
 [47535616 47566832 47598048 ... 48440880 48472096 48503312]
 [49092096 49124336 49156576 ... 50027056 50059296 50091536]]
True


In [53]:
a=np.array([[0,1],
            [2,3]])

b=np.array([[4,5], 
            [6,7]])

np.add(a,b)

array([[ 4,  6],
       [ 8, 10]])